<a href="https://colab.research.google.com/github/michalovsky/character_prediction/blob/master/character_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.callbacks import EarlyStopping as EarlyStopping
from tensorflow.python.client import device_lib
from tensorflow.keras.layers import Activation
import numpy as np
import os
import re
import time
import string

file = open('lotr.txt','rt')
text = file.read()
file.close()

chars_to_remove = ['…', '‚', '’', '‘', 'ó', '»', 'µ', '®', '«', '¥', '¤', '¢', '}', '{', '—', '_', '=', ';', '~', '`',
                   '#', '!', '\"', '#', '&', "\'", '(', ')', ',', '-', '.', '/', ':', '*', '[', ']', '\\', '|', '?', '–']

for char_to_remove in chars_to_remove:
  text = text.replace(char_to_remove, "")
  
text = text.lower()

#split text into words
# words = text.split(" ")
# print(*words)

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
    
# print('{')
# for char in char2idx:
#     print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
# print('}')
  
  
encoded = np.array([char2idx[ch] for ch in text])  
# text[:100]
encoded[:100]
  
  
# def one_with_zeros_encode(arr, n_labels):
#   one_zeros = np.zeros((np.multiply(*arr.shape)))
#   one_zeros[np.arrange(one_zeros.shape[0]), arr.flatten()] = 1.
#   one_zeros = one_zeros.reshape((*arr.shape, n_labels))

 







38 unique characters


array([ 0,  0,  0, 20, 25,  1, 12,  1, 19, 26, 23, 16,  1, 20, 25,  1, 31,
       19, 16,  1, 18, 29, 26, 32, 25, 15,  1, 31, 19, 16, 29, 16,  1, 23,
       20, 33, 16, 15,  1, 12,  1, 19, 26, 13, 13, 20, 31,  1, 25, 26, 31,
        1, 12,  1, 25, 12, 30, 31, 36,  1, 15, 20, 29, 31, 36,  1, 34, 16,
       31,  1, 19, 26, 23, 16,  1, 17, 20, 23, 23, 16, 15,  1, 34, 20, 31,
       19,  1,  0, 31, 19, 16,  1, 16, 25, 15, 30,  1, 26, 17,  1])